## Import Library

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_ta as ta

## Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
FAST_EMA_LENGTH = 5
LOW_EMA_LENGTH = 20
data = dataset.copy()
data["ema_f"] = ta.ema(data["Close"], length=FAST_EMA_LENGTH)
data["ema_f_shift"] = data["ema_f"].shift(1)
data["ema_l"] = ta.ema(data["Close"], length=LOW_EMA_LENGTH)
data["ema_l_shift"] = data["ema_l"].shift(1)
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma'] = data['above_ma'].rolling(150).sum()
data['total_below_ma'] = data['below_ma'].rolling(150).sum()
data['trend'] = data.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)

## Strategy

In [4]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at 20 pips
    sl_step = 2
    # Takeprofit at 60 pips(R/R = 1/3)
    tp_step = 6
    for i, row in prepared_data.iterrows():
        if 915 < 100*row.name.hour + row.name.minute < 1420:
            if has_open_deal:
                if type_open_deal == 'short':
                    # Stoploss
                    if row['High'] > entry_price + sl_step:
                        profit = -2
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['Low'] < entry_price - tp_step:
                            profit = 6
                            exit_price = entry_price - tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                            has_open_deal = False
                            type_open_deal = ''
                elif type_open_deal == 'long':
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -2
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['High'] > entry_price + tp_step:
                            profit = 6
                            exit_price = entry_price + tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
                            has_open_deal = False
                            type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    has_open_deal = True
                    type_open_deal = 'long'
        else:
            if 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -2:
                            profit = -2
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -2:
                            profit = -2
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit]
    return _trades

In [5]:
def cal_signal_01(row):
    signal = ''
    if row['trend'] == 'up' and row['ema_f_shift'] < row['ema_l_shift'] and row['ema_f'] > row['ema_l']:
        signal = 'long'
    elif row['trend'] == 'down' and row['ema_f_shift'] > row['ema_l_shift'] and row['ema_f'] < row['ema_l']:
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_01(r), axis=1)

In [6]:
data[data.signal != '']

,Open,High,Low,Close,Volume,ema_f,ema_f_shift,ema_l,ema_l_shift,ma_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,signal
Date,,,,,,,,,,,,,,,,
2018-08-13 10:40:00,944.9,945.0,943.7,943.7,936,944.198319,944.447479,944.319762,944.385000,944.415,0,1,NaN,NaN,down,short
2018-08-14 10:40:00,953.4,953.7,952.6,952.9,2550,954.366846,955.100268,954.747418,954.941883,955.690,0,1,NaN,NaN,down,short
2018-08-15 10:50:00,959.6,959.6,957.5,958.0,3980,959.769269,960.653904,960.130532,960.354798,960.385,0,1,NaN,NaN,down,short
2018-08-15 14:05:00,957.8,957.8,956.0,956.2,2322,957.384640,957.976960,957.497697,957.634297,957.045,0,1,NaN,NaN,down,short
2018-08-17 10:45:00,949.7,950.6,948.6,950.3,1984,950.745237,950.967856,950.757626,950.805797,952.105,0,1,71.0,79.0,down,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-25 13:55:00,1289.7,1289.8,1287.1,1287.9,4672,1289.534852,1290.352279,1289.592649,1289.770822,1289.080,0,1,43.0,107.0,down,short
2024-06-26 10:05:00,1288.9,1289.0,1288.0,1288.3,4135,1289.266968,1289.750452,1289.331357,1289.439921,1288.715,0,1,53.0,97.0,down,short
2024-06-27 09:30:00,1284.4,1284.4,1282.2,1282.8,5752,1284.432196,1285.248295,1284.809807,1285.021366,1284.960,0,1,50.0,100.0,down,short


In [7]:
trade02_result = trade_simulation(data)

In [8]:
trade02_result.Profit.sum()

252.90000000000066

In [9]:
trade02_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit
0,2018-08-13 10:40:00,943.7,2018-08-13 11:10:00,945.7,Short,-2.0
1,2018-08-14 10:40:00,952.9,2018-08-14 11:25:00,954.9,Short,-2.0
2,2018-08-15 10:50:00,958.0,2018-08-15 14:15:00,952.0,Short,6.0
3,2018-08-17 10:45:00,950.3,2018-08-17 11:05:00,952.3,Short,-2.0
4,2018-08-17 13:30:00,950.3,2018-08-17 14:25:00,947.0,Short,3.3
...,...,...,...,...,...,...
1302,2024-06-21 13:15:00,1316.8,2024-06-21 13:40:00,1318.8,Short,-2.0
1303,2024-06-25 13:55:00,1287.9,2024-06-25 14:00:00,1289.9,Short,-2.0
1304,2024-06-26 10:05:00,1288.3,2024-06-26 10:55:00,1282.3,Short,6.0
1305,2024-06-27 09:30:00,1282.8,2024-06-27 10:05:00,1284.8,Short,-2.0


In [10]:
# win rate
len(trade02_result[trade02_result.Profit > 0]) / (len(trade02_result[trade02_result.Profit < 0]) + len(trade02_result[trade02_result.Profit > 0]))

0.3287143956889915

In [11]:
this_month = trade02_result[(trade02_result.EntryTime > '2024-06-01 00:00:00') & (trade02_result.EntryTime < '2024-06-30 15:00:00')]
this_month.Profit.sum()

-7.400000000000091

In [12]:
this_year = trade02_result[(trade02_result.EntryTime > '2024-01-01 00:00:00') & (trade02_result.EntryTime < '2024-06-30 15:00:00')]
this_year.Profit.sum()

11.900000000000091

In [13]:
trade02_result.Profit.max()

11.599999999999909

In [14]:
trade02_result.Profit.min()

-2.0

In [15]:
trade02_result.Profit.mean()

0.19349655700076562

In [18]:
from datetime import date
from datetime import timedelta
from datetime import datetime
last_expire_date = date.today() - timedelta(days=90)
last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
_last_3months = trade02_result[trade02_result.EntryTime > last_expire_date]
_last_3months.Profit.sum()

-1.300000000000182